In [1]:
# Create MNIST data arrays
%run ./generate_mnist_dataset.ipynb

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Preprocessing MNIST training data...


 15%|█▌        | 1534/10000 [00:00<00:00, 15335.63it/s]

Preprocessing MNIST training data...


100%|██████████| 10000/10000 [00:00<00:00, 18451.44it/s]


In [2]:
# Load and prepare the data

import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from training_data import DataCollection
from PIL import Image
from matplotlib import pyplot as plt

def print_data_infos(data_train, data_test):
    print("Train data length: {0}".format(len(data_train.data)))
    print("Test data length: {0}".format(len(data_test.data)))
    print("Img Shape: {0}".format(data_train.data[0].shape))
    print("Number of Labels: {0}".format(data_train.no_labels))
    
data_all_train = DataCollection()
data_all_test = DataCollection(train=False)


print_data_infos(data_all_train, data_all_test)


  8%|▊         | 759/10000 [00:00<00:01, 7579.96it/s]

No training data for ). Skipping


100%|██████████| 10000/10000 [00:00<00:00, 365157.10it/s]


No training data for ). Skipping
Train data length: 72913
Test data length: 14750
Img Shape: torch.Size([1, 32, 32])
Number of Labels: 15


In [3]:
# Declare the network and some utilities

from torchvision import models
from torch.nn import Conv2d


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(train_loader, test_loader, model_name, print_step, num_classes=15, epochs=15):
    model = models.alexnet(num_classes=num_classes)
    model.features[0] = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    if torch.cuda.is_available():
        model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.8, 0.99), weight_decay=0.001)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        print("Epoch {0}".format(epoch))
        for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
            if torch.cuda.is_available():
                 x_train, y_train = x_train.to(device), y_train.to(device)
            optimizer.zero_grad()
            train_pred = model(x_train)
            loss = criterion(train_pred, y_train)
            loss.backward()
            optimizer.step()
            if step % print_step == 0:
                print('Loss: {}'.format(loss))
        
        acc = calc_accuracy(model, test_loader)
        print("Accuracy: {0}".format(acc))
        if acc > 98:
            torch.save(model.state_dict(), '{0}-{1}.ckpt'.format(model_name,acc))
    print("Accuracy: {0}".format(acc))
    torch.save(model.state_dict(), '{0}.ckpt'.format(model_name))

def calc_accuracy(model, test_loader):
    accuracies = []
    for idx, [x_test, y_test] in enumerate(tqdm(test_loader)):
        if torch.cuda.is_available():
            x_test, y_test = x_test.to(device), y_test.to(device)
        test_pred = model(x_test)
        accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
        accuracies.append(accuracy.item() if torch.cuda.is_available() else accuracy)
    return np.mean(accuracies)  

train_all_loader = DataLoader(data_all_train, batch_size=16, shuffle=True)
test_all_loader = DataLoader(data_all_test, batch_size=16, shuffle=False)



In [4]:
train(train_all_loader, test_all_loader, 'model-all-symbols', 500)

  0%|          | 4/4558 [00:00<01:58, 38.57it/s]

Epoch 0
Loss: 2.70410418510437


 11%|█         | 508/4558 [00:15<02:00, 33.53it/s]

Loss: 1.4603978395462036


 22%|██▏       | 1008/4558 [00:30<01:46, 33.47it/s]

...

Loss: 0.15450231730937958


 99%|█████████▉| 4506/4558 [02:14<00:01, 33.35it/s]

Loss: 0.22212405502796173


100%|██████████| 922/922 [00:02<00:00, 343.72it/s]

Accuracy: 94.16059808462147
Accuracy: 94.16059808462147


In [5]:
train_no_strokes = DataCollection(own_path='digits-plus-brackets', no_strokes=True, no_labels=13)
test_no_strokes = DataCollection(own_path='digits-plus-brackets', train=False, no_strokes=True, no_labels=13)
print_data_infos(train_no_strokes, test_no_strokes)

train_no_strokes_loader = DataLoader(train_no_strokes, batch_size=16, shuffle=True)
test_no_strokes_loader = DataLoader(test_no_strokes, batch_size=16, shuffle=False)

train(train_no_strokes_loader, test_no_strokes_loader, 'model-no_strokes', 500, num_classes=13)

  8%|▊         | 815/10000 [00:00<00:01, 8143.12it/s]

No training data for 0. Skipping
No training data for 1. Skipping
No training data for 2. Skipping
No training data for 3. Skipping
No training data for 4. Skipping
No training data for 5. Skipping
No training data for 6. Skipping
No training data for 7. Skipping
No training data for 8. Skipping
No training data for 9. Skipping
No training data for +. Skipping
No training data for -. Skipping
No training data for div. Skipping
No training data for brckts. Skipping
No training data for ). Skipping


100%|██████████| 10000/10000 [00:00<00:00, 369483.61it/s]


No training data for 0. Skipping
No training data for 1. Skipping
No training data for 2. Skipping
No training data for 3. Skipping
No training data for 4. Skipping
No training data for 5. Skipping
No training data for 6. Skipping
No training data for 7. Skipping
No training data for 8. Skipping
No training data for 9. Skipping
No training data for +. Skipping
No training data for -. Skipping
No training data for div. Skipping
No training data for brckts. Skipping
No training data for ). Skipping
Train data length: 60729
Test data length: 10081
Img Shape: torch.Size([1, 32, 32])
Number of Labels: 13


  0%|          | 4/3796 [00:00<01:39, 38.12it/s]

...

Loss: 0.0018676668405532837


 92%|█████████▏| 3506/3796 [01:44<00:08, 33.36it/s]

Loss: 0.04161544889211655


100%|██████████| 631/631 [00:01<00:00, 341.39it/s]


Accuracy: 98.44492868462757
Accuracy: 98.44492868462757


In [6]:
train_digits_plus= DataCollection(own_path='digits_plus', no_strokes=True, no_labels=11)
test_digits_plus = DataCollection(own_path='digits_plus', train=False, no_strokes=True, no_labels=11)
print_data_infos(train_digits_plus, test_digits_plus)

train_digits_plus_loader = DataLoader(train_digits_plus, batch_size=16, shuffle=True)
test_digits_plus_loader = DataLoader(test_digits_plus, batch_size=16, shuffle=False)

train(train_digits_plus_loader, test_digits_plus_loader, 'model-digits-plus', 500, num_classes=11)

  8%|▊         | 812/10000 [00:00<00:01, 8118.94it/s]

No training data for -. Skipping
No training data for div. Skipping
No training data for brckts. Skipping
No training data for ). Skipping


100%|██████████| 10000/10000 [00:00<00:00, 363817.29it/s]


No training data for -. Skipping
No training data for div. Skipping
No training data for brckts. Skipping
No training data for ). Skipping
Train data length: 65214
Test data length: 11685
Img Shape: torch.Size([1, 32, 32])
Number of Labels: 11


  0%|          | 5/4076 [00:00<01:45, 38.56it/s]

...

Loss: 0.00475563108921051


 98%|█████████▊| 4006/4076 [02:00<00:02, 33.31it/s]

Loss: 0.020339496433734894


100%|██████████| 731/731 [00:02<00:00, 339.77it/s]


Accuracy: 97.81976744186046
Accuracy: 97.81976744186046
